# Complex map data scraper
### Complexity lies in having to call the map data API many times from different coordinates, and in having a data limit per API call (1000 data json dictionaries/1000 mobile towers)
###### This is the map: https://geoportal.minetur.gob.es/VCTEL/vcne.do

In [1]:
import requests
import urllib.request, urllib.parse, urllib.error
import json
import sqlite3
import ssl
import time
from ast import literal_eval
import csv
import os

In [2]:
# bL_lat = -8.84892992
# bL_lon = 35.74657448
# tR_lat = 6.49954908
# tR_lon = 44.78973632
# These coordinates go from the bottom left of the spanish peninsula
# all the way streching out to Grenoble, France, in order to include
# the Balearic islands. The Canary islands are not included (to avoid excessive
# empty API calls across between the Canary Islands and Spain)

def get_data_from_api(bL_lat, bL_lon, tR_lat, tR_lon):
    headers = {
        'Connection': 'keep-alive',
        'sec-ch-ua': '"Google Chrome";v="93", " Not;A Brand";v="99", "Chromium";v="93"',
        'sec-ch-ua-mobile': '?0',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
        'sec-ch-ua-platform': '"Windows"',
        'Accept': '*/*',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://geoportal.minetur.gob.es/VCTEL/vcne.do',
        'Accept-Language': 'en-GB-oxendict,en;q=0.9,es-ES;q=0.8,es;q=0.7',
    }
    params = (
        ('idCapa', 'null'),
        ('bbox', f'{bL_lat}, {bL_lon}, {tR_lat}, {tR_lon}'), 
        ('zoom', '4'),
    )
    response = requests.get('https://geoportal.minetur.gob.es/VCTEL/infoantenasGeoJSON.do', headers=headers, params=params)
    return response.json()

In [3]:
# Create sqlite database to save data in there (so when code crashes, data is saved and so can keep going from there)
conn = sqlite3.connect('scrapedgeodatadb.sqlite', timeout= 20)
cur = conn.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS scrapedgeodatadb (id TEXT, data TEXT, UNIQUE(id, data))''') 

In [4]:
def scrape_data_from_given_area(start_lat, start_lon, end_lat, end_lon, lat_jump, lon_jump):
    current_lat = start_lat
    current_lon = start_lon
    start_time = time.time()
    while current_lon < end_lon:
        while current_lat < end_lat:
            a = str(get_data_from_api(current_lat, current_lon, current_lat + lat_jump, current_lon + lon_jump)) #SQLite
            b = str(str(current_lat) + ', ' +str(current_lon))
            print(current_lat, current_lon, end='\r')
            if a != '{}':
                cur.execute("INSERT INTO scrapedgeodatadb VALUES (?, ?)", [b,a]) #SQLite
                conn.commit() #SQLite
            current_lat += lat_jump
        pcg_cmn_dec = round((current_lon - start_lon+0.1) / (end_lon - start_lon),2)
        pcg_cmn_str = str(pcg_cmn_dec*100) + '%'
        current_time = time.time()
        elapsed_time_hrs = (current_time+0.001 - start_time)/60/60
        time_left_hrs = (elapsed_time_hrs / pcg_cmn_dec)
        print(pcg_cmn_str, str(round(time_left_hrs,2))+'hrs left')
        current_lat = start_lat
        current_lon += lon_jump
    return

In [5]:
start_lat = -8.84892992
start_lon = 35.846574479999994
end_lat = 6.49954908
end_lon = 43.69657447999955 #new end_lon (altura punto mas alto de españa, en galicia)
lat_jump = 0.05
lon_jump = 0.05
# jumps=0.5 --> 55,779 mobile towers
# jumps=0.4 --> 57,958 mobile towers
# jumps=0.35 --> 58,116 mobile towers
# jumps=0.3 --> 59,561 mobile towers
# jumps=0.2 --> 62,072 mobile towers
# jumps=0.1 --> 65,252 mobile towers
# jumps=0.05 --> 66,686 mobile towers

scrape_data_from_given_area(start_lat, start_lon, end_lat, end_lon, lat_jump, lon_jump)

ZeroDivisionError: float division by zero

In [30]:
# Copy db to new table without duplicates
cur.execute('''CREATE TABLE IF NOT EXISTS scrapedgeodatadb_2 (id TEXT, data TEXT, UNIQUE(id, data))''') 
cur.execute('''
INSERT INTO scrapedgeodatadb_2 SELECT DISTINCT id, data FROM scrapedgeodatadb;
''')

In [83]:
# Select data from database without duplicates, append all database rows together, 
# remove unwanted keys (there's 'type' and 'features', remove 'type', save only 'features'),
mobile_towers_data = []
cur.execute('SELECT data FROM scrapedgeodatadb_2')
for row in cur:
    mobile_towers_data.append(literal_eval(row[0])["features"])

[{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.53875, 36.058613]}, 'properties': {'Gis_Latitud': '36.058611', 'Gis_Longitud': '-5.538750', 'Gis_ID': 'ANDR5208A', 'Gis_Etiqueta': 'Estación de telefonía móvil', 'Gis_Estilo': 'vcne.estaciones', 'Gis_Codigo': 'ORANGE ESPAGNE, S.A.U. - ANDR5208A', 'Tipo': 'Estación de telefonía móvil', 'Código': 'ORANGE ESPAGNE, S.A.U. - ANDR5208A', 'Dirección': 'VP POLÍGONO 46 PARCELA 91, S/N. TARIFA, CÁDIZ', 'Detalle': '@@<url-aplicacion>/detalleEstacion.do?emplazamiento=ANDR5208A'}, 'id': 'ANDR5208A'}]
<class 'list'>


In [87]:
# flatten 'features' out (this will provide you with a seperate term for each mobile tower)
# print(len(flat_list)) to get number of mobile towers retrieved
flat_mobile_towers_data = [item for sublist in mobile_towers_data for item in sublist]
print(flat_mobile_towers_data[0:5])
print(len(flat_mobile_towers_data), "mobile towers in scraped data")

# You could even check if you have all the towers by checking if any cell in the database has 1000 towers 
# (number of times its says 'type': 'Feature' in a row in sqlite db)
# (if case exists, we can assume we would need more API calls in that area)

[{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.352475, 35.885765]}, 'properties': {'Gis_Latitud': '35.885764', 'Gis_Longitud': '-5.352475', 'Gis_ID': '102059', 'Gis_Etiqueta': 'Estación de telefonía móvil', 'Gis_Estilo': 'vcne.estaciones', 'Gis_Codigo': 'VODAFONE ESPAÑA, S.A. - 102059', 'Tipo': 'Estación de telefonía móvil', 'Código': 'VODAFONE ESPAÑA, S.A. - 102059', 'Dirección': 'VP POLÍGONO 1 PARCELA 247, S/N. CEUTA, CEUTA', 'Detalle': '@@<url-aplicacion>/detalleEstacion.do?emplazamiento=102059'}, 'id': '102059'}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-5.35255, 35.885746]}, 'properties': {'Gis_Latitud': '35.885747', 'Gis_Longitud': '-5.352550', 'Gis_ID': '5100067', 'Gis_Etiqueta': 'Estación de telefonía móvil', 'Gis_Estilo': 'vcne.estaciones', 'Gis_Codigo': 'TELEFONICA MOVILES ESPAÑA, S.A.U. - 5100067', 'Tipo': 'Estación de telefonía móvil', 'Código': 'TELEFONICA MOVILES ESPAÑA, S.A.U. - 5100067', 'Dirección': 'VP POLÍGONO 1 PARCELA

In [43]:
#Create and open csv file with the useful data retrieved
data_file = open('mobile_towers4.csv', 'w', newline='')
csv_writer = csv.writer(data_file)
 
count = 0
for data in flat_list:
    if count == 0:
        header = data['properties'].keys()
        csv_writer.writerow(header)
        count += 1
    csv_writer.writerow(data['properties'].values())
 
os.startfile("mobile_towers4.csv")
data_file.close()

In [16]:
os.remove("mobile_towers.csv")